# MediReportAssist - Google Colab Setup

This notebook sets up and runs the Patient Discharge Instructions Assistant in Google Colab.

## Features
- 📄 Document Upload (PDF, TXT, DOCX)
- 🔍 Natural Language Queries
- 🤖 RAG System with Hugging Face Transformers
- 💬 Query Manager for clarification
- 🌐 Public URL via ngrok


## Step 1: Install Dependencies


In [ ]:
# Install required packages
!pip install -q fastapi uvicorn[standard] python-multipart
!pip install -q chromadb sentence-transformers
!pip install -q pypdf2 python-docx
!pip install -q transformers torch accelerate
!pip install -q pyngrok

print("✅ All dependencies installed!")


## Step 2: Mount Google Drive (Optional but Recommended)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive for persistence
import os
os.makedirs('/content/drive/MyDrive/MediReportAssist', exist_ok=True)
%cd /content/drive/MyDrive/MediReportAssist

print("✅ Google Drive mounted!")


## Step 3: Upload Project Files

Upload these files using the file browser on the left:
- `app_colab.py`
- `rag_system_colab.py`
- `query_manager.py`
- `document_processor.py`
- `test_report_processor.py` (if needed)
- `static/` folder with `index.html`


## Step 4: Configure Settings


In [ ]:
import os

# Enable GPU (Runtime -> Change runtime type -> GPU)
USE_GPU = True  # Set to False if no GPU available

# Use 8-bit quantization to save memory
USE_8BIT = True

# Choose LLM model
LLM_MODEL = "gpt2"  # Start with small model

# Database path (use Drive for persistence)
DB_PATH = "/content/drive/MyDrive/MediReportAssist/data"

# Set environment variables
os.environ["USE_GPU"] = str(USE_GPU).lower()
os.environ["USE_8BIT"] = str(USE_8BIT).lower()
os.environ["LLM_MODEL"] = LLM_MODEL
os.environ["DB_PATH"] = DB_PATH

print(f"✅ Configuration:")
print(f"   GPU: {USE_GPU}")
print(f"   8-bit: {USE_8BIT}")
print(f"   LLM Model: {LLM_MODEL}")
print(f"   DB Path: {DB_PATH}")


## Step 5: Start Server with Public URL


In [ ]:

from pyngrok import ngrok
import uvicorn
from threading import Thread
import time

# Optional: Add your ngrok token here (uncomment and add token)
# ngrok.set_auth_token("your-ngrok-token-here")

# Start ngrok tunnel
tunnel = ngrok.connect(8000)
# Extract the public URL string from the tunnel object
public_url = tunnel.public_url
print(f"🌐 Public URL: {public_url}")
print(f"📱 Access your app at: {public_url}/static/index.html")
print(f"📚 API Docs: {public_url}/docs")
print(f"\n⚠️ IMPORTANT: Make sure to access the app using the ngrok URL above!")
print(f"   If you see 'Failed to fetch' or 'ERR_CONNECTION_REFUSED' errors:")
print(f"   1. You MUST access via: {public_url}/static/index.html")
print(f"   2. Do NOT use localhost:8000 - it won't work from your browser!")
print(f"   3. If you opened localhost, add this to the URL: ?api_url={public_url}")
print(f"   4. Or use the 'Set API URL' button that appears on the page\n")

# Start FastAPI server
def run_server():
    uvicorn.run("app_colab:app", host="0.0.0.0", port=8000, log_level="info")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(5)
print("✅ Server is running!")

## Step 6: Test the API (Optional)


In [ ]:
# Test the API endpoints
import requests
import time

# Wait a bit more to ensure server is ready
time.sleep(2)

# Test 1: Health check
try:
    response = requests.get(f"{public_url}/health", timeout=5)
    print("✅ Health Check:")
    print(response.json())
except Exception as e:
    print(f"❌ Health check failed: {e}")

# Test 2: Create a test file and upload
try:
    # Create a sample test file
    test_content = """DISCHARGE SUMMARY
    
Patient Name: Test Patient
Date: 2024-01-15

MEDICATIONS:
- Amoxicillin 500mg: Take twice daily (morning and evening) for 7 days
- Ibuprofen 200mg: Take as needed for pain, maximum 3 times per day

DIETARY RESTRICTIONS:
- Avoid spicy foods for 2 weeks
- No alcohol for 1 week
- Drink plenty of water

ACTIVITY RESTRICTIONS:
- No heavy lifting for 4 weeks
- Light walking is encouraged
- Avoid strenuous exercise

FOLLOW-UP:
- Schedule appointment in 2 weeks
- Contact doctor if fever > 101°F or severe pain
"""
    
    # Save test file
    test_file_path = "test_discharge.txt"
    with open(test_file_path, "w") as f:
        f.write(test_content)
    
    # Upload test file
    with open(test_file_path, "rb") as f:
        files = {"file": ("test_discharge.txt", f, "text/plain")}
        response = requests.post(f"{public_url}/upload", files=files, timeout=30)
    
    if response.status_code == 200:
        print("\n✅ Upload Test Successful:")
        print(response.json())
    else:
        print(f"\n❌ Upload Test Failed:")
        print(f"Status: {response.status_code}")
        print(f"Response: {response.text}")
        
except Exception as e:
    print(f"\n❌ Upload test failed: {e}")
    print("This is okay - you can test via the web interface instead!")
